# Create SQL database

This creates a sqlite DB from https://archive.analytics.mybinder.org.

In [1]:
import json
import urllib

import requests
import sqlite_utils

In [2]:
db = sqlite_utils.Database("binder-launches.db")

#db["binder"].drop()

In [3]:
index_json = "https://archive.analytics.mybinder.org/index.jsonl"

S = requests.Session()

# make it easier to grab the ref
def get_repo(spec):
    s = spec.rsplit("/", 1)[0]
    if s.endswith('.git'):
        s = s[:-4]
    return s


N = 0

for line in reversed(requests.get(index_json).text.split("\n")):
    if not line.strip():
        continue

    N += 1
    #if N > 365:
    #    break

    day = json.loads(line.strip())
    launches = S.get("https://archive.analytics.mybinder.org/{}".format(day['name'])).text.split("\n")
    
    launches_ = []
    for launch in launches:
        if launch:
            l = json.loads(launch)
            
            l["spec"] = urllib.parse.unquote(l["spec"])
            l['repo'] = get_repo(l['spec'])
            l["resolved_ref"] = l.get("ref", "")
            
            if l['provider'] not in ('GitHub', "Gist", "GitLab",
                                     "Git", 'Zenodo', 'Hydroshare',
                                     'Figshare', "Dataverse"):
                print(l)

            if l['provider'] == 'Hydroshare':
                l["org"] = l["ref"] = None
            elif l['provider'] in ('Zenodo', "Figshare", "Dataverse"):
                l["org"] = l["ref"] = l['repo'] = None
            else:
                if l["provider"] != "Git":
                    l['org'] = l['spec'].split("/", 1)[0]
                else:
                    l["org"] = None

                l['ref'] = l['spec'].rsplit("/", 1)[1]

            l.pop("schema")
            l.pop("status")
            
            launches_.append(l)

    db["binder"].insert_all(launches_)
    
db['binder'].enable_fts(["spec", "repo", "org"])

<Table binder (timestamp, version, provider, spec, ref, origin, repo, resolved_ref, org)>

In [4]:
db['binder'].create_index(["provider", "origin", "repo", "org", "ref", "resolved_ref", "timestamp"])

<Table binder (timestamp, version, provider, spec, ref, origin, repo, resolved_ref, org)>

In [5]:
db['binder'].optimize()

<Table binder (timestamp, version, provider, spec, ref, origin, repo, resolved_ref, org)>